## Collaborative Filtering in 10 Lines of Code

I was looking for some sample code for getting my hands dirty with some recommender system code, particularly around collaborative filtering. I came across this blog post [Collaborative Filtering in 9 Lines of Code](http://tungwaiyip.info/2012/Collaborative%20Filtering.html), which is based on the book Toby Seragan's book [Programming Collective Intelligence](http://shop.oreilly.com/product/9780596529321.do). 

This notebook is based on the orginal blog post. The code had to be adjusted to make it work with Python 3.

In [9]:
import numpy as np; import pandas as pd; from pandas import Series, DataFrame
rating = pd.read_csv('data/movie_rating.csv')
rp = rating.pivot_table(columns=['critic'], index=['title'],values='rating')
rp

critic,Claudia Puig,Gene Seymour,Jack Matthews,Lisa Rose,Mick LaSalle,Toby
title,,,,,,
Just My Luck,3.0,1.5,NaN,3.0,2.0,NaN
Lady in the Water,NaN,3.0,3.0,2.5,3.0,NaN
Snakes on a Plane,3.5,3.5,4.0,3.5,4.0,4.5
Superman Returns,4.0,5.0,5.0,3.5,3.0,4.0
The Night Listener,4.5,3.0,3.0,3.0,3.0,NaN
You Me and Dupree,2.5,3.5,3.5,2.5,2.0,1.0


The next step is to find the similarity score between the critics. The author Toby is used as an example. The book introduced a somewhat involving formula the Pearson correlation score. Turn out this is simply the correlation coefficient supported in most statistical packages. In Pandas, you can use corrwith() to calculate the correlation. A score close to 1 means their tastes are very similar. As you can see in the result below, Lisa Rose's taste is very similar to Toby but it is not so much with Gene Seymour.

In [169]:
rating_toby = rp['Toby']
sim_toby = rp.corrwith(rating_toby)
sim_toby

critic
Claudia Puig     0.893405
Gene Seymour     0.381246
Jack Matthews    0.662849
Lisa Rose        0.991241
Mick LaSalle     0.924473
Toby             1.000000
dtype: float64

To make recommendation for Toby, we calculate a rating of others weighted by the similarity. Note that we only need to calculate rating for movies Toby has not yet seen. The first line below filter out irrelevant data. It then assigns the similarity score and the weighted rating.

In [192]:
rating_c = rating.loc[rating_toby[rating.title].isnull().values & (rating.critic != 'Toby')]
rating_c.reset_index(drop=True, inplace=True)
rating_c_similarity = rating_c['critic'].map(sim_toby)

rating_c = rating_c.assign(similarity=rating_c_similarity, sim_rating=rating_c_similarity * rating_c.rating)
rating_c.head()

,critic,title,rating,sim_rating,similarity
0,Jack Matthews,Lady in the Water,3.0,1.988547,0.662849
1,Jack Matthews,The Night Listener,3.0,1.988547,0.662849
2,Mick LaSalle,Lady in the Water,3.0,2.773420,0.924473
3,Mick LaSalle,Just My Luck,2.0,1.848947,0.924473
4,Mick LaSalle,The Night Listener,3.0,2.773420,0.924473


Lastly we add up the score for each title using groupby(). We also normalize the score by dividing it with the sum of the weights. Base on other critics' similarity and their rating, we have made a movie recommendation for Toby. The number matches the result of the book.

In [193]:
sums = rating_c.groupby('title')['similarity', 'sim_rating'].sum()
recommendations = sums['sim_rating'] / sums['similarity']
recommendations.sort_index(ascending=False)

title
The Night Listener    3.347790
Lady in the Water     2.832550
Just My Luck          2.530981
dtype: float64

Putting it all together:

In [191]:
rating = pd.read_csv('data/movie_rating.csv')
rp = rating.pivot_table(columns=['critic'], index=['title'],values='rating')

rating_toby = rp['Toby']
sim_toby = rp.corrwith(rating_toby)

rating_c = rating.loc[rating_toby[rating.title].isnull().values & (rating.critic != 'Toby')]
rating_c_similarity = rating_c['critic'].map(sim_toby)
rating_c = rating_c.assign(similarity=rating_c_similarity, sim_rating=rating_c_similarity * rating_c.rating)

sums = rating_c.groupby('title')['similarity', 'sim_rating'].sum()
recommendations = sums['sim_rating'] / sums['similarity']
recommendations.sort_index(ascending=False)

title
The Night Listener    3.347790
Lady in the Water     2.832550
Just My Luck          2.530981
dtype: float64